In [1]:
import warnings
warnings.filterwarnings('ignore')
import math
import pandas as pd
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.cluster.hierarchy as sch
from seaborn.matrix import dendrogram
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from itertools import chain,  combinations

def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

wine = pd.read_csv('wine.csv')


In [2]:
y = wine['quality']
X = wine.drop(columns=['quality'])

#y = (y - y.mean())/y.std()
#X = (X - X.mean())/X.std()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae    = mean_absolute_error(y_test, y_pred)
print('MAE', round(mae,2))


MAE 0.42


In [13]:
synth = np.random.standard_normal(size=(1000,11))
synth = np.array(X_train)
ground = model.predict(synth)


In [ ]:
results = []
features = {i for i in range(11)}
for coalition in powerset(features):
    if len(coalition) >0:
        #X_train, X_test, y_train, y_test = train_test_split(synth[:, list(coalition)], ground, test_size=0.2, random_state=42)
        test_model = RandomForestRegressor(random_state=42)
        test_model.fit(np.array(X_train)[:,list(coalition)], y_train)
        y_pred = test_model.predict(np.array(X_test)[:,list(coalition)])
        mae = mean_absolute_error(y_test, y_pred)
        print(coalition, 'MAE', round(mae,2))
        results.append([coalition, mae])




(0,) MAE 0.67
(1,) MAE 0.68
(2,) MAE 0.66
(3,) MAE 0.67
(4,) MAE 0.66
(5,) MAE 0.65
(6,) MAE 0.66
(7,) MAE 0.65
(8,) MAE 0.68
(9,) MAE 0.68
(10,) MAE 0.63
(0, 1) MAE 0.69
(0, 2) MAE 0.67
(0, 3) MAE 0.64
(0, 4) MAE 0.68
(0, 5) MAE 0.7
(0, 6) MAE 0.65
(0, 7) MAE 0.57
(0, 8) MAE 0.7
(0, 9) MAE 0.68
(0, 10) MAE 0.61
(1, 2) MAE 0.66
(1, 3) MAE 0.62
(1, 4) MAE 0.64
(1, 5) MAE 0.65
(1, 6) MAE 0.64
(1, 7) MAE 0.56
(1, 8) MAE 0.67
(1, 9) MAE 0.66
(1, 10) MAE 0.57
(2, 3) MAE 0.63
(2, 4) MAE 0.64
(2, 5) MAE 0.66
(2, 6) MAE 0.62
(2, 7) MAE 0.56
(2, 8) MAE 0.63
(2, 9) MAE 0.66
(2, 10) MAE 0.61


In [15]:
def get_mae(S):
    if len(S) == 0: return 0
    for coalition, mae in results:
        if S == set(coalition):
            return 1 - mae

phi = 0
for i in features:
    phi_i = 0
    features_minus = features - {i}
    for S in powerset(features_minus):
        SS = set(S)
        gamma = math.factorial(len(SS)) * math.factorial(len(features) - len(SS) - 1) / math.factorial(len(features))
        phi_i += gamma * (get_mae(SS.union({i}))- get_mae(SS))
    print(i, ':' +  X.columns[i] + ':', round(phi_i,3))
    phi += phi_i
print(' :Total Reward: ', round(phi,3))

0 :fixed acidity: 0.049
1 :volatile acidity: 0.073
2 :citric acid: 0.059
3 :residual sugar: 0.062
4 :chlorides: 0.058
5 :free sulfur dioxide: 0.066
6 :total sulfur dioxide: 0.062
7 :density: 0.083
8 :pH: 0.053
9 :sulphates: 0.052
10 :alcohol: 0.096
 :Total Reward:  0.714


In [16]:
y2 = wine['quality']
X2 = wine.drop(columns=['quality'])